In [ ]:
"""
task1: 短期实习2
autor: 邵宇辉202083300563 20级气象学(8)班
time: 2023/03/22
"""
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt

import cartopy.crs as ccrs
import cartopy.feature as cfeature
from cartopy.io.shapereader import Reader
from cartopy.mpl.gridliner import LATITUDE_FORMATTER, LONGITUDE_FORMATTER
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter
from datetime import datetime
import os
import matplotlib.colors as mcolors
from scipy.stats import pearsonr
from statsmodels.stats.weightstats import ztest
from metpy.interpolate import inverse_distance_to_grid
from scipy.interpolate import griddata
from pykrige.ok import OrdinaryKriging
from eofs.standard import Eof
from math import radians, cos, sin, asin, sqrt
from tqdm import tqdm
from matplotlib import cm
from metpy.units import units
from metpy.constants import earth_avg_radius
import metpy.calc as mpcalc
import matplotlib.patches as patches
from cartopy.util import add_cyclic_point
from metpy.constants import earth_avg_radius
import xarray as xr
import netCDF4 as nc

In [ ]:
def creat_map(extent):  #底图绘制
    ##绘图区域
    proj = ccrs.PlateCarree()
    fig = plt.figure(figsize=(6,8), dpi= 300, facecolor='white')
    ax = fig.add_axes([0.1,0.1,0.6,0.8], projection= proj)
    # ##设置地图属性
    # states_provinces = cfeature.NaturalEarthFeature(
    #                     category='cultural',
    #                     name='admin_1_states_provinces_lines',
    #                     scale='50m',
    #                     facecolor='none')
    # ax.add_feature(states_provinces, edgecolor='k',linewidth= 0.6, zorder= 2)
    ax.add_feature(cfeature.LAND, facecolor='#D3D3D3')
    ax.add_feature(cfeature.OCEAN, facecolor='never')
    ax.add_feature(cfeature.COASTLINE.with_scale('50m'), linewidth= 0.6, zorder= 2)
    ax.add_feature(cfeature.BORDERS, linestyle='-',linewidth= 0.6,zorder= 2)
    ax.set_extent(extent, crs= ccrs.PlateCarree())
    return fig, ax

In [ ]:
data = xr.open_dataset(r'./hgt.mon.mean.1979_01-2019_01.nc')
hgt_clm = data.z.loc[:, 70:20, 40:140]/9.8              #1991-2020年500hPa位势高度气候态
lon = hgt_clm.longitude.values
lat = hgt_clm.latitude.values

# EOF模态分析
##1.计算纬度权重
coslat = np.cos(np.deg2rad(lat))
wgts = np.sqrt(coslat)[..., np.newaxis]
##2.创建EOF分解器
solver = Eof(np.array(hgt_clm), weights= wgts)
eof = solver.eofsAsCorrelation(neofs= 3)  #此处eofs值即为需要的空间模态数
pc = solver.pcs(npcs=3, pcscaling= 1)     #方差
var = solver.varianceFraction(neigs= 3)

In [ ]:
##3.绘图-空间第1模态
for i in np.arange(0,3):
    fig, ax = creat_map(extent= [40, 140, 20, 70])
    # 设置ticks
    ax.set_xticks([60, 90, 120], crs=ccrs.PlateCarree())
    ax.set_yticks([25, 45, 65], crs=ccrs.PlateCarree())
    ax.xaxis.set_major_formatter(LongitudeFormatter(zero_direction_label= True))
    ax.yaxis.set_major_formatter(LatitudeFormatter())
    C = ax.contourf(lon, lat, eof[i], transform= ccrs.PlateCarree(), extend='both', levels= np.linspace(-1,1,100),cmap='RdBu_r', zorder=0)
    cbar_ax = fig.add_axes([0.1,0.33,0.6,0.025])
    fig.colorbar(C,
                orientation= 'horizontal', cax= cbar_ax,
                ticks= np.linspace(-1,1,5+1)
                )
    fig.savefig(r'../PIC/P2_1979_2019_EOF空间第'+str(i)+'模态_'+str(var[i])+'_.svg',format='svg', dpi=300, pad_inches= 0.1, bbox_inches= 'tight')
    print(var[i])

In [ ]:
##3.绘图-空间第1模态
import proplot as pplt
for i in np.arange(0,3):
    # 时间模态
    years = np.arange(1979, 2019+1)
    fig, ax = pplt.subplots(
                            refheight = 1.4,
                            refwidth= 4,
                            dpi= 300,
    )
    ax.axhline(0, ls="--", lw= 1)
    ax.plot(years, pc[:,i], color= '#8b2671', marker='o',markersize=2, lw=1.3)
    ax.format(
            ylim=(-3,3),
            xlim=(1977, 2021),
            yticks= 1.5,
            xticks= 5,
            linewidth= 1.1,
            titlesize= 14,
            ytickminor=False,
    )
    fig.savefig(r'../PIC/P2_1979_2019_EOF时间第'+str(i)+'模态.svg',format='svg', dpi=300, pad_inches= 0.1, bbox_inches= 'tight')